In [ ]:
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_validate, train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error
from scipy.signal import savgol_filter
from scipy.optimize import minimize
from scipy.stats import skew, kurtosis
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class BaselineRegressor:
    """
    Baseline regressor, which calculates the mean value of the target from the training
    data and returns it for each testing sample.
    """
    def __init__(self):
        self.mean = 0

    def fit(self, X_train: np.ndarray, y_train: np.ndarray):
        self.mean = np.mean(y_train, axis=0)
        self.classes_count = y_train.shape[1]
        return self

    def predict(self, X_test: np.ndarray):
        return np.full((len(X_test), self.classes_count), self.mean)


class SpectralCurveFiltering():
    """
    Create a histogram (a spectral curve) of a 3D cube, using the merge_function
    to aggregate all pixels within one band. The return array will have
    the shape of [CHANNELS_COUNT]
    """

    def __init__(self, merge_function = np.mean):
        self.merge_function = merge_function

    def __call__(self, sample: np.ndarray):

        sample = sample.reshape(sample.shape[0],-1)


        band_mean = np.mean(sample, axis=1)
        band_std = np.std(sample, axis=1)
        band_var = np.var(sample, axis=1)
        band_skew = skew(sample, axis=1)
        band_kurt = kurtosis(sample, axis=1)
        band_area = np.array([sample.shape[1]]*sample.shape[0])
        return np.array([band_mean, band_std, band_var, band_skew, band_kurt, band_area ]).T

In [ ]:
def load_data(directory: str):
    """Load each cube, reduce its dimensionality and append to array.

    Args:
        directory (str): Directory to either train or test set
    Returns:
        [type]: A list with spectral curve for each sample.
    """
    data = []
    filtering = SpectralCurveFiltering()
    all_files = np.array(
        sorted(
            glob(os.path.join(directory, "*.npz")),
            key=lambda x: int(os.path.basename(x).replace(".npz", "")),
        )
    )
    for file_name in all_files:
        with np.load(file_name) as npz:

            arr = np.ma.MaskedArray(**npz)
        arr = filtering(arr)
        data.append(arr)
    return np.array(data)



def load_gt(file_path: str):
    """Load labels for train set from the ground truth file.
    Args:
        file_path (str): Path to the ground truth .csv file.
    Returns:
        [type]: 2D numpy array with soil properties levels
    """
    gt_file = pd.read_csv(file_path)
    labels = gt_file[["P", "K", "Mg", "pH"]].values
    return labels


X_train = load_data("train_data/train_data")
y_train = load_gt("train_data/train_gt.csv")
X_test = load_data("test_data")

print(f"Train data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")


Train data shape: (1732, 150, 6)
Test data shape: (1154, 150, 6)


In [ ]:
np.save('stats_train_bands', X_train)
np.save('stats_test_bands', X_test)
np.save('stats_train_label', y_train)

In [ ]:
# Define the Savitzky-Golay second-derivative function
def sg_sd(x, window_length=7, polyorder=2):
    # Apply the Savitzky-Golay filter to smooth the spectra
    smoothed = savgol_filter(x, window_length, polyorder, axis=1)
    # Calculate the second-derivative of the smoothed spectra
    sd = np.gradient(np.gradient(smoothed, axis=1), axis=1)
    return sd

In [ ]:
# Load the formatted data
train_bands = np.load('stats_train_bands.npy').transpose((0,2,1))  # transform into shape ([Lambda1 mean,...]*150 , [Lambda1 std,...]*150,... [....])
test_bands = np.load('stats_test_bands.npy').transpose((0,2,1))
train_elements = np.load('stats_train_label.npy')

In [ ]:
train_area = train_bands[:,-1,:].mean(axis=1).reshape(-1,1) # the last last column is the area dim
test_area = test_bands[:,-1,:].mean(axis=1).reshape(-1,1)

train_bands = train_bands[:,:-1,:].reshape(train_bands.shape[0],-1) # reshape into 2D matrix without the area dim
test_bands = test_bands[:,:-1,:].reshape(test_bands.shape[0],-1)

train_bands = np.hstack((train_bands, train_area)) # add the area column as the last column into the formatted 2D array
test_bands = np.hstack((test_bands, test_area))

# using red band 630 to 690; blue band 450 - 520; green band 520 - 600 and NIR band 760 - 900 (nm)
min_wav = 462
red_start, red_end = int((630 - min_wav)/3.2), int((690 - min_wav)/3.2)
blue_end = int((520 - min_wav)/3.2)
green_start, green_end = int((520 - min_wav)/3.2), int((600 - min_wav)/3.2)
nir_start, nir_end = int((760 - min_wav)/3.2), int((900 - min_wav)/3.2)


train_bands.shape, test_bands.shape

((1732, 751), (1154, 751))

In [ ]:
# creating color and NIR bands using the MEAN only.

train_blue = train_bands[:,0:blue_end].mean(1)
train_green = train_bands[:,green_start:green_end].mean(1)
train_red = train_bands[:,red_start:red_end].mean(1)
train_nir = train_bands[:,nir_start:nir_end].mean(1)


test_blue = test_bands[:,0:blue_end].mean(1)
test_green = test_bands[:,green_start:green_end].mean(1)
test_red = test_bands[:,red_start:red_end].mean(1)
test_nir = test_bands[:,nir_start:nir_end].mean(1)

train_nvdi = (train_nir - train_red)/(train_nir + train_red) # vegetation index
train_pri = (train_green - train_blue)/(train_green + train_blue)
train_evi = 2.5*(train_nir - train_red)/(train_nir + 6*train_red - 7.5*train_blue +1) # enhanced vegetation index
train_savi = (train_nir - train_red)/(train_nir + train_red + 0.5)*(1 + 0.5) # soil index
train_ratio1 = train_red/train_blue
train_ratio2 = train_red/train_green
train_ratio3 = train_blue/train_green
train_ratio4 = train_blue/train_green
train_ratio5 = train_green/train_red
train_ratio6 = train_green/train_blue

test_nvdi = (test_nir - test_red)/(test_nir + test_red)
test_pri = (test_green - test_blue)/(test_green + test_blue)
test_evi = 2.5*(test_nir - test_red)/(test_nir + 6*test_red - 7.5*test_blue +1)
test_savi = (test_nir - test_red)/(test_nir + test_red + 0.5)*(1 +0.5)
test_ratio1 = test_red/test_blue
test_ratio2 = test_red/test_green
test_ratio3 = test_blue/test_green
test_ratio4 = test_blue/test_green
test_ratio5 = test_green/test_red
test_ratio6 = test_green/test_blue



color_train = np.array([train_blue, train_green, train_red, train_nir, train_nvdi, train_evi, train_savi, train_ratio1,train_ratio2,train_ratio3,train_ratio4,train_ratio5,train_ratio6 ]).T
color_test = np.array([test_blue, test_green, test_red, test_nir, test_nvdi, test_evi, test_savi, test_ratio1,test_ratio2,test_ratio3,test_ratio4,test_ratio5,test_ratio6]).T

In [ ]:
# join all the created features
train_band_features = np.hstack((train_bands, color_train))
test_band_features = np.hstack((test_bands, color_test))

#  only the MEAN which is from (0 to 149th column) we'll be used and savgol filter with 2nd derivate applied to smoothen the signal
train_band_features[:,0:150] = sg_sd(train_band_features[:,0:150], window_length=3, polyorder=1) # window length and polyorder were tuned
test_band_features[:,0:150] = sg_sd(test_band_features[:,0:150], window_length=3, polyorder=1)

# scale the features
train_band_features = (train_band_features - train_band_features.mean(0))/train_band_features.std(0)
test_band_features = (test_band_features - test_band_features.mean(0))/test_band_features.std(0)

In [ ]:
# Defining the parameters of the models, all were tuned using hyperopt optimization.

forest_params = {'n_estimators': 203,'max_depth': 78,'max_leaf_nodes': 348,'min_samples_split': 6,'min_samples_leaf': 4,
                 'min_weight_fraction_leaf': 0.0010047252525281973,'min_impurity_decrease': 0.0020686506542306835,'max_features': 0.9995836445306979,
                 'random_state':42, 'n_jobs':-1
                }

etr_params = {'n_estimators': 102,'min_samples_split': 8,'min_samples_leaf': 2,'min_weight_fraction_leaf': 0.001467859820666249,
              'min_impurity_decrease': 0.10117651856072411,'max_features': 0.96464832151798,'random_state':42, 'n_jobs':-1
             }

knn_params = {'n_neighbors': 54, 'leaf_size': 134, 'p': 5, 'weights':'distance', 'n_jobs':-1}

lgb_params = {'boosting': 'gbdt','num_iterations': 173,'learning_rate': 0.01807409716203315,
              'tree_learner': 'serial','num_leaves': 539,'max_depth': 6,'n_estimators': 236,'subsample_for_bin': 16269,'min_split_gain': 0.13801317023253418,
              'min_child_weight': 0.6547238467990038,'min_child_samples': 5,'reg_alpha': 0.00015187650791215833,'reg_lambda': 0.0771509874666758,
              'bagging_freq': 1,'bagging_fraction': 0.9202173572091822,'feature_fraction': 0.9721966537259545,'random_state':42, 'n_jobs':-1,'verbose':-1,
             }

In [ ]:
# To make the predictions we'll use 4 fold cv and make predictions. oof predictions will also be used to finding weights to add the model outputs

def oof_model(features, target, test_features, models, cv=4): # models is list of  tuples [(model_class, model_name, model_params),..., (...)]
    test_predictions = []  # to store the predictions from different model
    model_oofs = []
    model_ytrues = []  # just stores the copies of y_true
    print("Info from oof models")
    for model, name, param in models:
        oof_predictions = np.zeros(target.shape)
        y_true = np.zeros(target.shape)  # stores the target values
        kf = KFold(n_splits=cv, shuffle=True, random_state=2023)
        test_pred = []
        for train_idx, valid_idx in kf.split(features):
            train_x, valid_x = features[train_idx], features[valid_idx]
            train_y, valid_y = target[train_idx], target[valid_idx]
            if 'lgb' in name:
                new_model = MultiOutputRegressor(model(**param))
            else:
                new_model = model(**param)
            new_model.fit(train_x, train_y)
            pred_y = new_model.predict(valid_x)

            y_true[valid_idx] = valid_y
            oof_predictions[valid_idx] = pred_y

            test_pred.append(new_model.predict(test_features))

        model_oofs.append(oof_predictions)
        model_ytrues.append(y_true)

        # print the errors and r2
        rmse = np.sqrt(mean_squared_error(oof_predictions, y_true))
        r2 = r2_score(oof_predictions, y_true)
        # print(f"{name} rmse : {rmse} -- {name} r2 : {r2}")
        test_predictions.append(np.mean(test_pred,axis=0))
    print()
    return test_predictions, model_oofs, model_ytrues


In [ ]:
# define the models for oof prediction
models = [(RandomForestRegressor, 'forest', forest_params),
          (ExtraTreesRegressor,  'etr', etr_params),
          (KNeighborsRegressor, 'knn', knn_params),
          (LGBMRegressor, 'lgb', lgb_params)]


In [ ]:
# for observation, we'll split the data into train and validation
train_bands, valid_bands, train_target, valid_target = train_test_split(train_band_features, train_elements, test_size=0.3, random_state=42)

# get the model predictions from oof_models
valid_predictions_oof, train_oofs, train_Ys = oof_model(train_bands, train_target, valid_bands, models)

Info from oof models
[LightGBM] [Warning] bagging_fraction is set=0.9202173572091822, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9202173572091822
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.9721966537259545, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9721966537259545
[LightGBM] [Warning] bagging_fraction is set=0.9202173572091822, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9202173572091822
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.9721966537259545, colsample_bytree=1.0 will be ignor

In [ ]:
# lets check the errors of each model for each of the outputs

model_order = ['forest','etr','knn','lgb']
for i, model_name in enumerate(model_order):
    oof_pred =valid_predictions_oof[i]
    for j,tar in enumerate(['P2O5','Potassium','Magnesium','pH']):
        oof_rmse = np.sqrt(mean_squared_error(oof_pred[:,j],valid_target[:,j]))
        print(f"{model_name} - {tar} rmse : {oof_rmse}")

# print mean model prediction and the errors
mean_pred = np.array([train_elements.mean(0)]*len(valid_target))
for i,tar in enumerate(['P2O5','Potassium','Magnesium','pH']):
    mean_rmse = np.sqrt(mean_squared_error(mean_pred[:,i],valid_target[:,i]))
    print(f"Mean model - {tar}  rmse : {mean_rmse}")

forest - P2O5 rmse : 24.30678466350733
forest - Potassium rmse : 55.1070960440621
forest - Magnesium rmse : 37.88845891312706
forest - pH rmse : 0.24204854850833096
etr - P2O5 rmse : 24.446533197901775
etr - Potassium rmse : 54.68751019076692
etr - Magnesium rmse : 38.04175478095468
etr - pH rmse : 0.23960802389719812
knn - P2O5 rmse : 24.692258313266873
knn - Potassium rmse : 57.72475193935712
knn - Magnesium rmse : 38.98221668720845
knn - pH rmse : 0.24941461331799802
lgb - P2O5 rmse : 24.345389932211912
lgb - Potassium rmse : 54.8646523234527
lgb - Magnesium rmse : 37.74227988897312
lgb - pH rmse : 0.2361300877404062
Mean model - P2O5  rmse : 25.066388715950136
Mean model - Potassium  rmse : 59.11923439038279
Mean model - Magnesium  rmse : 39.251741731664985
Mean model - pH  rmse : 0.25723848447413805


In [ ]:
# from above we can see that some models perform particulary well on specific target,
# Ex:- Lightgbm has lowest mse among all the models for pH;
# ExtraTreesRegressor has lowest mse among all the models for Potassium;
# RandomForest has lowest for Magnesium, etc.

# Therefore i decided to use weights not only for model prediction as whole but also each target.
# In total there will be (Num. of model X Num. of target) weights.

In [ ]:
# Combine the predictions into a single function
def combined_predictions(weights, predictions):
    predictions = np.array(predictions).transpose(1,2,0) # to shape (num of samples, num of target, num of model)
    custom_weights = np.array(weights).reshape(predictions.shape[1], predictions.shape[2]) # to shape (num of target, num of model)
    pred  = []
    for i, w_row in enumerate(custom_weights[:]):
        pred.append(np.sum(w_row*predictions[:,i,:], axis=1))
    return np.array(pred).T

# Define the objective function to minimize (mean squared error + r2)
def objective(weights, predictions, actual):
    pred = combined_predictions(weights, predictions)
    mse = mean_squared_error(actual, pred)
    return mse

def weighted_prediction(model_oofs, model_true, num_target=4):
    # Initialize weights
    initial_weights = np.ones(len(model_oofs)*num_target)

    # Optimize the weights
    result = minimize(objective, initial_weights, args=(model_oofs, model_true), method='L-BFGS-B')

    # Get the optimized weights
    optimized_weights = result.x

    return optimized_weights

In [ ]:
# We'll be using the train_oofs and train_targets to find the optimal weights for
model_true  = np.mean(train_Ys, 0) # taking mean changes nothing, there are just copies the train_target values

# get the optimized weights
optimized_weights = weighted_prediction(train_oofs, model_true)

In [ ]:
# every consecutive four weights represents the weights to different model in the order (forest, etr, knn, and lgb)
# as you can see the the last weight correspond to the lgb weight for pH and it has the highest among all and as we saw earlier lgb gave lowest MSE,
# similarly the first weight corresponds to the forest model prediction of P2O5 and as we saw earlier this again gets the highest weight, so on and so forth.
optimized_weights

array([ 0.71135591,  0.11715665, -0.17557001,  0.33810638,  0.21707926,
        0.33871198, -0.04713769,  0.48723512, -0.06140541,  0.11978116,
        0.17859078,  0.76470645,  0.24959966,  0.2497377 ,  0.25102225,
        0.25042097])

In [ ]:
# Lets compare the predictions

oof_valid_pred = np.mean(valid_predictions_oof,0)
mean_valid_pred = np.array([train_elements.mean(0)]*len(valid_target))

print("RMSE with oof predictions ",np.sqrt(mean_squared_error(oof_valid_pred, valid_target)))
print("RMSE with mean model predictions ",np.sqrt(mean_squared_error(mean_valid_pred, valid_target)))

RMSE with oof predictions  35.557506676076464
RMSE with mean model predictions  37.63035051248978


In [ ]:
# lets make predictions using the weights and see the results using the valid_prediction_oof and valid_prediction_cv
weighted_oof_pred = combined_predictions(optimized_weights, valid_predictions_oof)

print("RMSE with weighted oof predictions ",np.sqrt(mean_squared_error(weighted_oof_pred, valid_target)))
print("RMSE with mean model predictions ",np.sqrt(mean_squared_error(mean_valid_pred, valid_target)))

RMSE with weighted oof predictions  35.292709539050925
RMSE with mean model predictions  37.63035051248978


In [23]:
# Now that we have our weights lets make prediction for the test data, for this we'll use the whole training data and make the prediction

test_prediciton, _, _ = oof_model(train_band_features, train_elements, test_band_features, models)

weighted_test_prediction = combined_predictions(optimized_weights, test_prediciton)

Info from oof models
[LightGBM] [Warning] bagging_fraction is set=0.9202173572091822, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9202173572091822
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.9721966537259545, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9721966537259545
[LightGBM] [Warning] bagging_fraction is set=0.9202173572091822, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9202173572091822
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.9721966537259545, colsample_bytree=1.0 will be ignor

In [24]:
# Before submitting we need to normalize the values
mean_norm = train_elements.mean(0)
weighted_test_prediction /= mean_norm

In [25]:
# load the sample submission
sample_sub = pd.read_csv('SampleSubmission.csv.csv')
model_sub = pd.DataFrame(data = weighted_test_prediction, columns=["P", "K", "Mg", "pH"]).reset_index()


model_sub.columns = ['sample_index', 'P', 'K', 'Mg', 'pH']
model_sub = model_sub.melt(id_vars=['sample_index'], value_vars=['P', 'K', 'Mg', 'pH'])
model_sub.sort_values('sample_index',inplace=True)

model_sub['sample_index'] = model_sub['sample_index'].astype('str')+"_"+model_sub['variable']
model_sub.drop('variable', inplace=True, axis=1)
model_sub.columns = ['sample_index', 'Target']
model_sub.reset_index(inplace=True, drop=True)

sample_sub.drop('Target',axis=1,inplace=True)
sample_sub = sample_sub.merge(model_sub, on='sample_index', how='left')
print(sample_sub.head())

  sample_index    Target
0          0_P  0.961311
1          0_K  0.961130
2         0_Mg  0.974144
3         0_pH  1.012643
4          1_P  0.989327


In [26]:
# save the file
sample_sub.to_csv('Submission.csv', index=False)